# `CLOUD.jl` - 2D linear advection example

In [1]:
using CLOUD, OrdinaryDiffEq, TimerOutputs

Define the physical problem

In [2]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

Set discretization parameters

In [3]:
M = 4
p = 8
p_map = 4
form = WeakConservationForm()
strategy = ReferenceOperator()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [4]:
reference_approximation = ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGLQuadrature(),
    facet_quadrature_rule=LGLQuadrature(),
    mapping_degree=p_map, N_plot=50,
    operator_algorithm=DefaultOperatorAlgorithm())

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), strategy,
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [5]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy);

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

CLOUD_reset_timer!()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

Thread 1
 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            16.9s /  69.8%           5.06GiB /  74.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 rhs!                   4.81k    11.8s  100.0%  2.45ms   3.74GiB  100.0%   816KiB
   apply operators      77.0k    10.4s   88.4%   135μs   2.40GiB   64.2%  32.7KiB
     volume terms       77.0k    9.16s   77.7%   119μs   1.98GiB   52.8%  26.9KiB
     mass matrix solve  77.0k    649ms    5.5%  8.43μs    108MiB    2.8%  1.44KiB
     facet terms        77.0k    307ms    2.6%  3.99μs    147MiB    3.8%  1.96KiB
   gather ext state     77.0k    100ms    0.8%  1.30μs    402MiB   10.5%  5.34KiB
   ext

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [6]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
4.0524482426575444e-17
Conservation (initial/final/diff):
[2.9456931685545307e-11][2.9456931685545307e-11][0.0]
Energy (initial/final/diff):
[0.12499999985776973][0.12499999985776973][0.0]


Write output to VTK file

In [7]:
postprocess_vtk(spatial_discretization, string(results_path,"solution.vtu"), last(sol.u));